In [2]:
import guidance
import os
import diskcache
import pathlib
import requests
import html
from urllib.parse import urlparse
import urllib.parse
import io
import html
import html.parser

curr_dir = './'# pathlib.Path(__file__).parent.resolve()
_bing_cache = diskcache.Cache(f"{curr_dir}/../bing.diskcache")

with open(os.path.expanduser('~/.bing_api_key'), 'r') as file:
    subscription_key = file.read().replace('\n', '')

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"
    # search_url =
#     search_term = "Who are the founders of OpenAI?"

    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        if params_key not in _bing_cache or "webPages" not in _bing_cache[params_key]:
            # print("SEARCHING BING")
            response = requests.get(search_url, headers=headers, params=params)
            response.raise_for_status()
            _bing_cache[params_key] = response.json()
        # print(type(_bing_cache[params_key]))
        # print(list(_bing_cache[params_key]))
        # print(_bing_cache[params_key])
        # result_batch = _bing_cache[params_key]["webPages"]["value"]

        data = _bing_cache[params_key]["webPages"]["value"]
        for r in data:
            r["snippet_text"] = strip_tags(r["snippet"])

        search_results.extend(data)

    return search_results

In [3]:
def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]

In [4]:
# llm = guidance.llms.OpenAI("text-davinci-003") # could also specify default temperature here
llm = guidance.llms.OpenAI("text-davinci-003") # could also specify default temperature here

In [11]:
top_snippets('openai founders')

[{'title': 'OpenAI - Wikipedia',
  'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'title': 'About - OpenAI',
  'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'title': 'Ilya Sutskever | Stanford HAI',
  'snippet': 'Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads researc

In [43]:
top_snippets('facebook stop price')

[{'title': 'How do I cancel a Subscription? | Facebook Help Center',
  'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
 {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
  'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
 {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
  'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Stati

In [74]:
def is_search(completion):
    return '<search>' in completion
def search(query):
    return top_snippets(query, n=3)
prompt = guidance('''<|im_start|>system
Assistant is a large language model trained by OpenAI.
Whenever its response depends on any factual information, it searches the web by using the function <search>query</search> before responding to the user.
For example, it might start its response with
<search>Who are the founders of OpenAI</search>
After which the top snippets from search will be displayed.
<|im_end|>{{#each history}}
<|im_start|>{{this.user}}
{{this.utterance}}
<|im_end|>{{/each}}
<|im_start|>user
{{query}}
<|im_end|>
<|im_start|>assistant{{#block hidden=True}}
{{generate "answer" stop="</search>"}}{{/block}}{{#if (is_search answer)}}</search>{{#each (search answer)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{generate "more"}}
{{/if}}''', llm=llm)
query = "What is Facebook's stock price right now?"
prompt(is_search=is_search, search=search, query=query,
       history=[
    {'user': 'user', 'utterance': 'What is truth?'}, 
    {'user': 'assistant', 'utterance': '42'}
    ])

<|im_start|>system
Assistant is a large language model trained by OpenAI.
Whenever its response depends on any factual information, it searches the web by using the function <search>query</search> before responding to the user.
For example, it might start its response with
<search>Who are the founders of OpenAI</search>
After which the top snippets from search will be displayed.
<|im_end|>
<|im_start|>user
What is truth?
<|im_end|>
<|im_start|>assistant
42
<|im_end|>
<|im_start|>user
What is Facebook's stock price right now?
<|im_end|>
<|im_start|>assistant
<search>What is Facebook's stock price</search>
<result>
Meta Platforms, Inc. (META) Stock Price, News, Quote &amp; History - Yahoo ...
Meta Platforms, Inc. (META) Stock Price, News, Quote & History - Yahoo Finance U.S. markets closed +239.31(+2.14%) +32.59(+1.87%) (+0.57%) Meta Platforms, Inc. (META) NasdaqGS - NasdaqGS Real...
</result>
<result>
News | FACEBOOK Stock Price Today | Analyst Opinions - Insider
Facebook Inc (A) Cert Deposito Arg Repr 1-8 Sh -A- Stock 2,926.00 ARS +5.50 +0.19% Official Close 3/9/2023 BUE Add to watchlist News Analyst Data Analyst Opinions Insider Activity Dividend...
</result>
<result>
Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...
Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $179.51 -2.18 (-1.2%) Updated Mar 10, 2023 1W - 3.1% 1M + 3.08% 3M + 56.49% 1Y - 8.04% META Price $179.51 Fair Value Price $184.94 Market Cap $465.40B 52 Week Low
</result>
The current stock price of Facebook is $179.51.